In [172]:
from textblob import classifiers
import pandas as pd
from newsapi import NewsApiClient

In [173]:
PATH1 = 'data/SentimentData/amazon_cells_labelled.txt'
PATH2 = 'data/SentimentData/imdb_labelled.txt'
PATH3 = 'data/SentimentData/yelp_labelled.txt'

amazon_data = pd.read_csv(PATH1, sep='\t', header=None, names=['sentence', 'sentiment'])
imdb_data = pd.read_csv(PATH2, sep='\t', header=None, names=['sentence', 'sentiment'])
yelp_data = pd.read_csv(PATH3, sep='\t', header=None, names=['sentence', 'sentiment'])

In [174]:
total_data = amazon_data
total_data = total_data.append(imdb_data, ignore_index=True)
total_data = total_data.append(yelp_data, ignore_index=True)

In [175]:
train_data_full = [tuple(x) for x in total_data.values]
train_data = train_data_full[:2000]
print(len(train_data))

2000


In [176]:
print('training NLP classifier...')
classifier = classifiers.NaiveBayesClassifier(train_data)
classifier.train()
print('done')

training NLP classifier...
done


In [177]:
print('classifying...')
prob_dist = classifier.prob_classify("Stocks are terrible")
print('result')
print(prob_dist.prob(0))

classifying...
result
0.6754882030498195


In [178]:
classifier.accuracy(train_data_full[2000:])

0.7593582887700535

In [179]:
key = '8b8428b3603940ef8485d39a6eb526f2'
news = NewsApiClient(api_key=key)

companies = {'Synchrony Financial': [],
            '3M': [],
            'Bayer': [],
            'Honeywell': []}

ARTICLE_COUNT = 5

for company in companies:
    query = company + " News"
    all_articles = news.get_everything(q=query, from_param='2019-02-01',to='2019-02-02', language='en', sort_by='relevancy')
    article_data = all_articles['articles']
    for i in range(ARTICLE_COUNT):
        try:
            titlePlusSummary = article_data[i]['title'] + '. ' + article_data[i]['description']
            companies[company].append(titlePlusSummary)
        except:
            break

In [180]:
# Open the choosen news articles and extract the title
title_to_polarity = {}
net_company_scores = {}
for company in companies:
    net_company_scores[company] = 0
    count = 0
    for summary in companies[company]:
        try:
            prob_1 = classifier.prob_classify(summary).prob(1)
            title_to_polarity[summary] = prob_1
            net_company_scores[company] += prob_1
        except:
            continue
    if count:
        net_company_scores[company] /= count

print(net_company_scores)

{'Synchrony Financial': 1.3697317812082912, '3M': 1.5567991879540197, 'Bayer': 3.6251427205105813, 'Honeywell': 2.003248803329603}
